LOAD THE DATA SET

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('grammar_correction_pairs.csv')

# Prepare training data
incorrect_sentences = df['incorrect_sentence'].values
correct_sentences = df['correct_sentence'].values

TOKENIZATION

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(incorrect_sentences) + list(correct_sentences))

# Convert text to sequences
X = tokenizer.texts_to_sequences(incorrect_sentences)
y = tokenizer.texts_to_sequences(correct_sentences)

# Pad sequences to make them the same length
X = pad_sequences(X, padding='post')
y = pad_sequences(y, padding='post')

DEFINE THE MODEL

In [3]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, RepeatVector, TimeDistributed

# Define the model
model = Sequential()

# Define embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256, input_length=X.shape[1]))

# Encoder
model.add(LSTM(256))

# Decoder
model.add(RepeatVector(X.shape[1]))  # Repeat the context vector
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tokenizer.word_index) + 1, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


TRAIN THE MODEL

In [4]:
# Train the model
model.fit(X, y, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 27s 124ms/step - accuracy: 0.3672 - loss: 3.0709 - val_accuracy: 0.4629 - val_loss: 4.0673
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 39s 115ms/step - accuracy: 0.7980 - loss: 0.8388 - val_accuracy: 0.4239 - val_loss: 4.6955
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 41s 113ms/step - accuracy: 0.8238 - loss: 0.6603 - val_accuracy: 0.4768 - val_loss: 4.6292
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 22s 121ms/step - accuracy: 0.8832 - loss: 0.4588 - val_accuracy: 0.5253 - val_loss: 4.3920
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 22s 121ms/step - accuracy: 0.9140 - loss: 0.3253 - val_accuracy: 0.5280 - val_loss: 4.4326
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 42s 125ms/step - accuracy: 0.9237 - loss: 0.2595 - val_accuracy: 0.5537 - val_loss: 4.2018
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 41s 123ms/step - accuracy: 0.9300 - loss: 0.2247 - val_accuracy: 0.5575 - val_loss: 4.4543
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 21s 115ms/step - accuracy: 0.9475 - loss: 0

INFERENCE

In [5]:
# Function to predict the corrected sentence
def correct_grammar(input_sentence):
    seq_input = tokenizer.texts_to_sequences([input_sentence])
    padded_input = pad_sequences(seq_input, padding='post', maxlen=X.shape[1])

    pred = model.predict(padded_input)

    # Convert prediction to words
    pred_sentence = ' '.join([tokenizer.index_word.get(idx, '') for idx in pred[0].argmax(axis=-1)])
    return pred_sentence

USER INPUT

In [6]:
# Get user input and correct grammar
user_input = input("Enter a sentence with possible grammar errors: ")
corrected_sentence = correct_grammar(user_input)
print("Corrected sentence:", corrected_sentence)

Enter a sentence with possible grammar errors: මම යන්නෙමු
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
Corrected sentence: මම යන්නෙමි     


Grammar Correction Function for Paragraphs:

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to split paragraph into sentences by period (.)
def split_paragraph_into_sentences(paragraph):
    # Split the paragraph by punctuation mark (.)
    sentences = paragraph.split(".")

    # Clean up sentences (remove empty sentences after split)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

# Function to correct a single sentence (using the trained model)
def correct_grammar_for_sentence(input_sentence, tokenizer, model, max_input_length):
    # Tokenize and pad the input sentence
    seq_input = tokenizer.texts_to_sequences([input_sentence])
    padded_input = pad_sequences(seq_input, padding='post', maxlen=max_input_length)

    # Predict the corrected sentence
    pred = model.predict(padded_input)

    # Convert prediction to words
    pred_sentence = ' '.join([tokenizer.index_word.get(idx, '') for idx in pred[0].argmax(axis=-1)])
    return pred_sentence

# Function to correct grammar in an entire paragraph
def correct_grammar_in_paragraph(paragraph, tokenizer, model, max_input_length):
    # Step 1: Split the paragraph into sentences
    sentences = split_paragraph_into_sentences(paragraph)

    # Step 2: Correct each sentence
    corrected_sentences = [correct_grammar_for_sentence(sentence, tokenizer, model, max_input_length) for sentence in sentences]

    # Step 3: Join the corrected sentences back into a paragraph
    corrected_paragraph = '. '.join(corrected_sentences) + '.' if corrected_sentences else ''

    return corrected_paragraph

In [8]:
# Example usage
# Assuming you have the model and tokenizer already loaded
user_paragraph = "මම යන්නෙමු. අපි යනවා. මම යවන්නෙමි. මම යවන්නෙහි. මම ගියාය. අපි යනවාලා."
corrected_paragraph = correct_grammar_in_paragraph(user_paragraph, tokenizer, model, X.shape[1])

print("Original Paragraph: ", user_paragraph)
print("Corrected Paragraph: ", corrected_paragraph)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Original Paragraph:  මම යන්නෙමු. අපි යනවා. මම යවන්නෙමි. මම යවන්නෙහි. මම ගියාය. අපි යනවාලා.
Corrected Paragraph:  මම යන්නෙමි     . මම යැවෙමි     . මම යමි     . මම යවන්නෙමි     . මම ගියෙමි     . මම යැවෙමි     .
